In [ ]:
import pandas as pd
import numpy as np
import os
import json
import requests
import re


In [ ]:
# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)

In [ ]:
# Load all data
frames = []
for filename in os.listdir("./data/newssources"):
    frame = pd.read_json("./data/newssources/"+filename, lines=True)
    frames.append(frame)
df_raw = pd.concat(frames)

In [ ]:
# %store df_raw
%store -r df_raw

In [ ]:
len(df_raw["id"].unique())

In [ ]:
# data size
print(len(df_raw))
df_raw.head()

In [ ]:
df_raw.columns

### Filtered for specific columns

In [ ]:
# Filtered for specific columns
col_names = ['id', 'date', 'updated', 'title', 'caption', 'expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'statistics', 'account']
df_cols = df_raw[col_names]

In [ ]:
df_cols[0:1]

### Normalizing Account Column Values as Multiple Columns

In [ ]:
## Expanding statistics and account columns
df_acc = df_cols.join(pd.json_normalize(df_cols['account']).add_prefix('account_'))
df_acc[0:1]

In [ ]:
print(len(df_raw["id"].unique()))
print(len(df_acc["id"].unique()))

In [ ]:
df_acc.columns

In [ ]:
df_acc = df_acc.drop(columns=['account', 'account_profileImage', 'account_platform', 'account_platformId'])
df_acc[0:1]

### Normalizing Statistics Column Values as Multiple Columns

In [ ]:
stat_col = df_acc.statistics.apply(lambda x: x['actual'])
df_stat = df_acc.join(pd.json_normalize(stat_col).add_prefix('statistics_'))
df_stat[0:1]

In [ ]:
df_stat.columns

In [ ]:
df_stat = df_stat.drop(columns=['statistics'])

In [ ]:
df_stat[0:1]

### Adding rows based on 'expandedLinks' column values. One row each for every 'original' (unique) key in the dictionary.

In [ ]:
df_stat['expandedLinks'].values[0]

#### rows with no expandedLinks

In [ ]:
len(df_raw[df_raw['expandedLinks'].isnull()])

In [ ]:
df_raw[df_raw['expandedLinks'].isnull()][['expandedLinks', 'link', 'postUrl']]

In [ ]:
df_exp = df_stat.explode('expandedLinks').reset_index()

In [ ]:
print(len(df_exp))
print(len(df_exp.groupby('id').count()))

In [ ]:
df_exp_count = df_exp[['expandedLinks', 'id']].groupby('id').count()

In [ ]:
df_exp_count.sort_values('expandedLinks', ascending=False)[2600:2605]

In [ ]:
df_raw[df_raw['id'] == 111599342312]['expandedLinks']

In [ ]:
expanded_link_original = df_exp.expandedLinks.apply(lambda x: x['expanded'] if not pd.isnull(x) else None)

In [ ]:
expanded_link_original[0:4]

In [ ]:
df_exp = df_exp.rename(columns={'expandedLinks': 'expandedLinksRaw'})

In [ ]:
# df_final = df_stat.join(df_exp).drop(columns='expandedLinks')
df_final = df_exp.join(expanded_link_original)

In [ ]:
df_final.columns

In [ ]:
df_final[df_final['id'] == 111599342312][['id', 'expandedLinksRaw', 'expandedLinks']]

In [ ]:
# verifying that 'original' url has been extracted from allnon-null expandedLinks 

lost = df_final[df_final['expandedLinks'].isnull() & df_final['expandedLinksRaw'].notnull()][['expandedLinks', 'expandedLinksRaw']]
lost

In [ ]:
final_cols = ['id', 'date', 'updated', 'title', 'caption', 'expandedLinks', 'link', 'postUrl',
       'subscriberCount', 'score', 'account_id', 'account_name',
       'account_handle', 'account_subscriberCount', 'account_url',
       'account_accountType', 'account_pageAdminTopCountry',
       'account_verified', 'statistics_likeCount', 'statistics_shareCount',
       'statistics_commentCount', 'statistics_loveCount',
       'statistics_wowCount', 'statistics_hahaCount', 'statistics_sadCount',
       'statistics_angryCount', 'statistics_thankfulCount',
       'statistics_careCount']
news_df = df_final[final_cols].drop_duplicates()

In [ ]:
news_df.head()

In [ ]:
len(news_df)

In [ ]:
%store news_df

#### No. of Unique Ids/Links

In [ ]:
unique_posts_by_id = len(news_df["id"].unique())
unique_posts_by_links = len(news_df["link"].unique())
print("Unique Post Ids: ", unique_posts_by_id)
print("Unique Post Links: ", unique_posts_by_links)

In [ ]:
news_df[news_df['id'] == 111599342312]

#### Saving news_df to csv

In [ ]:
# news_df.to_csv('news_df.csv')

#### Loading NewsGuardTech Ratings

In [ ]:
with open("./data/ratings/newsguard-ratings.json") as f:
    ratings = f.read()
ratings_data = json.loads(ratings)

In [ ]:
ratings_df = pd.DataFrame(ratings_data.items(), columns=['domain', 'rating'])

In [ ]:
ratings_df

In [ ]:
# exp = r'(?i)(.*h?t?tps?://)?(\w*(-->>|:))?(www\.)?([A-Za-z_0-9.-]+).*'
exp = r'(?i)(.*h?t?tps?://)?(www\.)?([A-Za-z_0-9.-]+).*'

In [ ]:
news_df['domain'] = news_df.expandedLinks.str.extract(exp, expand=True)[2]

In [ ]:
news_df[['expandedLinks', 'domain']]

In [ ]:
# QC
len(news_df[news_df['expandedLinks'].notnull() & news_df['domain'].isnull()])

In [ ]:
# intersection of newsguardtech api domains and crowdtangle api domains
ratings_domains_set = set(ratings_df['domain'].unique())
ct_domains_set = set(news_df['domain'].unique())

In [ ]:
len(ct_domains_set.difference(ratings_domains_set))

todo:
pre-tasks: correct domain column 
1. include ratings column
2. include credible (t/f) column

In [ ]:
import pandasql as ps

In [ ]:
invalid_expanded_links = ps.sqldf("select id, expandedLinks, domain from news_df where domain not like '%.%'")
# news_df[~news_df["domain"].str.contains('.', na=False)][['id', 'expandedLinks', 'domain']]

In [ ]:
invalid_expanded_links.to_csv('invalid_expanded_links.csv')

In [ ]:
len(invalid_expanded_links.id.unique())

In [ ]:
# bad data
df_raw[df_raw['id'] == 111663444165]

In [ ]:
invalid_exp_data = df_raw[df_raw['id'].isin(invalid_expanded_links.id.to_list())][['id', 'expandedLinks']]
invalid_exp_data.to_csv('invalid_exp_data.csv')

In [ ]:
news_df[(news_df['id'].isin(invalid_expanded_links.id.to_list())) & (news_df['domain'].str.contains('.'))][['id', 'expandedLinks', 'domain']]

In [ ]:
news_df[news_df['id'] == 111677826151]

#### Expanding short urls to obtain the correct domain

In [ ]:
## wherever rating = NaN, get expandedLink - parse it for domain using requests
to_get_expanded_url = news_df[news_df.domain.isin(['trib.al', 'bit.ly', 'tinyurl.com'])][['id', 'expandedLinks', 'domain']]

In [ ]:
len(to_get_expanded_url)

In [ ]:
short_url_list = to_get_expanded_url.expandedLinks.unique().tolist()

In [ ]:
len(short_url_list)

In [ ]:
def extract_domain(url):
    exp = r'(?i)(.*h?t?tps?://)?(www\.)?([A-Za-z_0-9.-]+).*'
    try:
        search = re.search(exp, url, re.IGNORECASE)
        return search.groups()[2]
    except Exception as e:
        print(f'Exception for {url} - {e}')
    

In [ ]:
def get_extended_url(url):
    extended_url = ''
    domain = ''
    try:
        extended_url = requests.head(url).headers['location']
    except (KeyError, TypeError) as e:
        print(f'Key/Type Exception for {url} - {e}')
    except requests.exceptions.MissingSchema as e:
        print(f'Missing schema for {url}')
        corrected_url =  'http://'+url
        extended_url = requests.head(corrected_url).headers['location']
    except Exception as e:
        print(f'Exception for {url} - {e}')
    domain = extract_domain(extended_url)    
    if domain and domain in ['trib.al', 'bit.ly', 'tinyurl.com']:
        extended_url = requests.head(extended_url).headers['location']
        domain = extract_domain(extended_url)
    return extended_url, domain

In [ ]:
def cleanup_url(url):
    pos = url.find('https')
    if url[-1] == '.':
        url = url[0:-1]
    if pos > 0:
        return url[pos:]
    return url

In [ ]:
# short_expanded_url_map = {}

In [ ]:
get_extended_url('bit.ly/2UeUZGn')

In [ ]:
# for url in short_url_list:
#     if url not in short_expanded_url_map.keys():
#         expanded_url, domain = get_extended_url(cleanup_url(url))
#         if expanded_url != '':
#             short_expanded_url_map[url] = [expanded_url, domain]

In [ ]:
# %store short_expanded_url_map
%store -r short_expanded_url_map

In [ ]:
data = [[short_url, val[1]]
        for short_url, val in short_expanded_url_map.items()]
corrected_domain_df = pd.DataFrame(data, columns=['expandedLinks', 'corrected_domain'])

In [ ]:
corrected_domain_df

In [ ]:
get_extended_url('https://bit.ly/3nPNfYa?cc=539458b52df6684692eb0107760bd294')

In [ ]:
corrected_domain_df[corrected_domain_df.corrected_domain.isin(['trib.al', 'bit.ly', 'tinyurl.com'])][['expandedLinks', 'corrected_domain']]

In [ ]:
news_updated_domain_df = pd.merge(news_df,corrected_domain_df,how='left',left_on=['expandedLinks'],right_on=['expandedLinks'])

In [ ]:
news_updated_domain_df[news_updated_domain_df.corrected_domain.notnull()][['domain', 'corrected_domain']]

In [ ]:
# temp['domain'] = temp[['domain', 'corrected_domain']].apply(lambda x: x[1] if pd.notnull(x[1]) else x[0])
news_updated_domain_df['domain'] = np.where(news_updated_domain_df['corrected_domain'].notnull(), news_updated_domain_df['corrected_domain'], news_updated_domain_df['domain'])

In [ ]:
news_updated_domain_df[news_updated_domain_df.corrected_domain.notnull()][['domain', 'corrected_domain']]

In [ ]:
news_updated_domain_df = news_updated_domain_df.drop(columns=['corrected_domain'])

In [ ]:
links404 = news_updated_domain_df[news_updated_domain_df.domain.isin(['trib.al', 'bit.ly', 'tinyurl.com'])][['id', 'expandedLinks', 'domain']]

In [ ]:
len(links404)

### Joining news_updated_domain_df with ratings_df 

In [ ]:
news_ratings_df = pd.merge(news_updated_domain_df,ratings_df,how='left',left_on=['domain'],right_on=['domain'])

In [ ]:
news_ratings_df['rating'] = news_ratings_df['rating'].fillna(-1)

In [ ]:
nonna_ratings = news_ratings_df[news_ratings_df.rating.notnull()].domain.unique()

In [ ]:
len(nonna_ratings)

In [ ]:
nan_ratings = news_ratings_df[news_ratings_df.rating.isnull()].domain.unique()

In [ ]:
len(nan_ratings)

In [ ]:
news_ratings_df[(news_ratings_df['rating'].isnull())][['domain']].drop_duplicates().sort_values('domain', ascending=True).to_csv('no_ratings.csv', index=False)

In [ ]:
len(news_ratings_df[news_ratings_df.rating.isnull()])

In [ ]:
len(news_ratings_df[news_ratings_df.rating.notnull()])

In [ ]:
news_ratings_df[news_ratings_df['rating'] == -1][['id', 'domain']].groupby('domain').count().sort_values('id', ascending=False).to_csv('domain_rating_null.csv')

In [ ]:
news_ratings_df.head()

In [ ]:
news_ratings_df['credible'] = news_ratings_df.rating.apply(lambda x: True if x >= 60 else False)

In [ ]:
news_ratings_df.head()

In [ ]:
news_ratings_df.to_csv('news_ratings_df.csv', index=False)